In [6]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('aurora',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: C:\Users\lppap\Documents\master\scalable_ML\id2223-project
HopsworksSettings initialized!


## Imports

In [7]:
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)
import hopsworks
from mlfs.aurora import util
import json

import warnings
warnings.filterwarnings("ignore")

FEATURE_VIEW_NAME = "aurora_fv"
FEATURE_VIEW_VERSION = 1   # change to 2 for solar wind, 1 no solar wind
MODEL_NAME = "aurora_xgboost"
MODEL_VERSION = FEATURE_VIEW_VERSION

HORIZONS = [1, 2, 3, 4, 5]

RANDOM_STATE = 42
TEST_RATIO = 0.2

## Hopsworks login & Connect

In [8]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

2026-01-04 16:52:01,437 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-04 16:52:01,445 INFO: Initializing external client
2026-01-04 16:52:01,447 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-04 16:52:02,959 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279154


In [4]:
# Retrieve feature groups
geomagnetic_fg = fs.get_feature_group( name="geomagnetic_daily", version=1)
weather_fg = fs.get_feature_group(name="sweden_weather_daily", version=1)
solar_fg = fs.get_feature_group(name="nasa_omni_daily", version=1)

## Feature View creation

In [67]:
# Select and join features for training
OldQuery = (
    geomagnetic_fg.select(
        ["date", "kp1", "kp2", "kp3", "kp4", "kp5", "kp6", "kp7", "kp8", "ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8", "ap"]
    ).join(
        weather_fg.select([
            "cloud_cover_mean",
            "precipitation_sum",
            "sunshine_duration"
        ]),
        on=["date"]
    ))

NewQuery = (
    geomagnetic_fg.select(
        ["date", "kp1", "kp2", "kp3", "kp4", "kp5", "kp6", "kp7", "kp8", "ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8", "ap"]
    ).join(
        weather_fg.select([
            "cloud_cover_mean",
            "precipitation_sum",
            "sunshine_duration"
        ]),
        on=["date"]
    ).join(
        solar_fg.select([
            "vsw_lag1", "vsw_lag2",
            "bz_lag1", "bz_lag2",
            "pressure_lag1",
            "bz_3d_mean", "bz_7d_min",
            "vsw_3d_mean", "pressure_3d_max",
            "vbz_neg"
        ]),
        on=["date"]
    )
)


## Create the feature view

In [72]:
feature_view_old = fs.get_or_create_feature_view(
    name="aurora_fv",
    description="Geomagnetic and weather features for aurora visibility prediction",
    version= 1,
    labels=["ap"],
    query=OldQuery,
)

feature_view_new = fs.get_or_create_feature_view(
    name="aurora_fv",
    description="Geomagnetic, weather and solar features for aurora visibility prediction",
    version= 2,
    labels=["ap"],
    query=NewQuery,
)


In [19]:
feature_view = fs.get_feature_view(
    name=FEATURE_VIEW_NAME,
    version=FEATURE_VIEW_VERSION
)

In [20]:
X_base, y_base = feature_view.training_data(
    description="Training data for multi-horizon aurora models"
)

X_base["date"] = pd.to_datetime(X_base["date"], utc=True)

print("Date range in training data:")
print(X_base["date"].min(), "→", X_base["date"].max())

print(X_base.shape)
print(X_base.columns)
X_base.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.15s) 
Date range in training data:
2020-01-01 00:00:00+00:00 → 2026-01-01 00:00:00+00:00
(2192, 20)
Index(['date', 'kp1', 'kp2', 'kp3', 'kp4', 'kp5', 'kp6', 'kp7', 'kp8', 'ap1',
       'ap2', 'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8', 'cloud_cover_mean',
       'precipitation_sum', 'sunshine_duration'],
      dtype='object')


,date,kp1,kp2,kp3,kp4,kp5,kp6,kp7,kp8,ap1,ap2,ap3,ap4,ap5,ap6,ap7,ap8,cloud_cover_mean,precipitation_sum,sunshine_duration
0,2020-02-17 00:00:00+00:00,1.333,0.333,1.333,0.667,1.000,1.333,3.000,3.000,5.0,2.0,5.0,3.0,4.0,5.0,15.0,15.0,80.333336,1.1,14275.975586
1,2020-03-15 00:00:00+00:00,0.000,0.333,0.333,1.667,1.333,1.000,1.667,1.667,0.0,2.0,2.0,6.0,5.0,4.0,6.0,6.0,88.583336,1.7,469.042084
2,2020-04-06 00:00:00+00:00,1.000,1.000,0.333,0.000,1.000,0.333,0.000,0.000,4.0,4.0,2.0,0.0,4.0,2.0,0.0,0.0,67.708336,0.0,39656.863281
3,2020-04-10 00:00:00+00:00,0.000,0.667,1.000,1.333,1.000,2.333,1.000,0.333,0.0,3.0,4.0,5.0,4.0,9.0,4.0,2.0,67.166664,0.6,44831.945312
4,2020-04-15 00:00:00+00:00,3.000,2.667,1.667,1.000,1.000,0.667,1.000,1.333,15.0,12.0,6.0,4.0,4.0,3.0,4.0,5.0,75.250000,0.8,19225.437500


## Feature engineering - Lagged geomagnetic features

In [21]:
# Ensure data is sorted by time
order = X_base["date"].sort_values().index
X_base = X_base.loc[order].reset_index(drop=True)
y_base = y_base.loc[order].reset_index(drop=True)

ap = y_base["ap"]

# Lagged Ap features
for lag in [1, 2, 3]:
    X_base[f"ap_lag_{lag}"] = ap.shift(lag)
    

# Lagged Kp features (daily mean + max are most informative)
kp_cols = [f"kp{i}" for i in range(1, 9)]
X_base["kp_mean"] = X_base[kp_cols].mean(axis=1)
X_base["kp_max"] = X_base[kp_cols].max(axis=1)

for lag in [1, 2, 3]:
    X_base[f"kp_mean_lag_{lag}"] = X_base["kp_mean"].shift(lag)
    X_base[f"kp_max_lag_{lag}"] = X_base["kp_max"].shift(lag)

# Drop non-feature columns
#X = X_base.drop(columns=["date"])


### Training loop — one model per horizon

In [22]:
# Build binary targets for all horizons
AP_THRESHOLD = 15
MAX_HORIZON = 5

y_targets = {
    h: (ap.shift(-h) >= AP_THRESHOLD).astype("int32")
    for h in range(1, MAX_HORIZON + 1)
}

X = X_base.drop(columns=["date"])

In [23]:
# Train / test split (time-aware,
split_idx = int((1 - TEST_RATIO) * len(X))

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

train_mask = X_train.index
test_mask  = X_test.index

## 5 days target models

Design rules:

- One horizon = one model
- Same features for all horizons
- Only the target shifts
- Same train/test split for all horizons
- One model name per horizon 

In [24]:
import pandas as pd
import seaborn as sns

def feature_importance_plot(model, model_dir):
    # Get feature importance by gain
    importance = model.get_booster().get_score(importance_type="gain")
    
    # Convert to DataFrame
    imp_df = (
        pd.DataFrame(importance.items(), columns=["feature", "gain"])
          .sort_values("gain", ascending=False)
          .head(20)
    )
    
    # Plot
    plt.figure(figsize=(6, 4))
    sns.barplot(
        data=imp_df,
        x="gain",
        y="feature",
        color="steelblue"
    )
    plt.title(f"Top 20 Feature Importances (t+{h})")
    plt.xlabel("Gain")
    plt.ylabel("Feature")
    
    plt.tight_layout()
    plt.savefig(f"{model_dir}/feature_importance_gain_tplus{h}.png")
    plt.close()

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, accuracy_score,
    precision_score, recall_score,
    f1_score, confusion_matrix
)
import os
import joblib
import seaborn as sns

MODEL_VERSION = 1
mr = project.get_model_registry()

for h in range(1, MAX_HORIZON + 1):
    print(f"\n🚀 Training model for t+{h}")

    y_train = y_targets[h].iloc[train_mask]
    y_test  = y_targets[h].iloc[test_mask]

    # Safety checks
    assert len(X_train) == len(y_train)
    assert len(X_test) == len(y_test)

    pos = y_train.sum()
    neg = len(y_train) - pos
    scale_pos_weight = (neg / max(pos, 1))

    model = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="auc",
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )

    model.fit(X_train, y_train)

    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred  = (y_proba >= 0.5).astype(int)

    metrics = {
        "roc_auc": float(roc_auc_score(y_test, y_proba)),
        "accuracy": float(accuracy_score(y_test, y_pred)),
        "precision": float(precision_score(y_test, y_pred, zero_division=0)),
        "recall": float(recall_score(y_test, y_pred, zero_division=0)),
        "f1": float(f1_score(y_test, y_pred, zero_division=0)),
        "horizon_days": h,
        "ap_threshold": AP_THRESHOLD,
        "feature_view_version": FEATURE_VIEW_VERSION
    }

    cm = confusion_matrix(y_test, y_pred)
    print("Confusion matrix:")
    print(cm)
    print("Metrics:", metrics)
    
    # Save Model
    model_dir = "aurora_model"
    os.makedirs(model_dir, exist_ok=True)

    # Creating confusion matrix
    plt.figure(figsize=(4, 3))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        cbar=False
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title(f"Confusion Matrix (t+{h})")
    
    plt.tight_layout()
    plt.savefig(f"{model_dir}/confusion_matrix_tplus{h}.png")
    plt.close()
    
    model_path = os.path.join(model_dir, f"model_h{h}.pkl")

    feature_importance_plot(model, model_dir)
    joblib.dump(model, model_path)

    model_name = f"{MODEL_NAME}_h{h}"
    model_description = (
        f"Binary aurora classifier (Ap >= 15) predicting t+{h} days ahead "
    )

    registered_model = mr.python.create_model(
        name= model_name,
        version = MODEL_VERSION,
        description= model_description,
        metrics=metrics
    )
    
    registered_model.save(model_dir)

    print(f"✅ Model {model_name} saved and registered")


🚀 Training model for t+1
Confusion matrix:
[[228  67]
 [ 55  89]]
Metrics: {'roc_auc': 0.7624529190207155, 'accuracy': 0.7220956719817767, 'precision': 0.5705128205128205, 'recall': 0.6180555555555556, 'f1': 0.5933333333333333, 'horizon_days': 1, 'ap_threshold': 15, 'feature_view_version': 1}

🚀 Training model for t+2
Confusion matrix:
[[240  55]
 [102  42]]
Metrics: {'roc_auc': 0.6184322033898304, 'accuracy': 0.642369020501139, 'precision': 0.4329896907216495, 'recall': 0.2916666666666667, 'f1': 0.3485477178423237, 'horizon_days': 2, 'ap_threshold': 15, 'feature_view_version': 1}

🚀 Training model for t+3
Confusion matrix:
[[239  56]
 [115  29]]
Metrics: {'roc_auc': 0.5231638418079096, 'accuracy': 0.6104783599088838, 'precision': 0.3411764705882353, 'recall': 0.2013888888888889, 'f1': 0.25327510917030566, 'horizon_days': 3, 'ap_threshold': 15, 'feature_view_version': 1}

🚀 Training model for t+4
Confusion matrix:
[[241  54]
 [126  18]]
Metrics: {'roc_auc': 0.5139830508474577, 'accura

In [33]:
##TODO: MOVE THIS APP TO BEFORE THE DATE IS REMOVE FROM THE DF

'''
test_start = pd.Timestamp("2025-05-01", tz="UTC")

train_mask = X["date"] < test_start
test_mask = X["date"] >= test_start

X_train = X.loc[train_mask].drop(columns=["date"])
X_test  = X.loc[test_mask].drop(columns=["date"])
'''


In [38]:
model_name = "aurora_xgboost_3day"
model_description = (
    "XGBoost classifier predicting probability of aurora-favorable "
    "geomagnetic conditions occurring within the next 5 days, "
    "based on geomagnetic indices (Kp, Ap) and weather features."
)

In [39]:
import os
import joblib

model_dir = "aurora_model"
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, "model.pkl")
joblib.dump(model, model_path)


['aurora_model\\model.pkl']

In [51]:
registered_model = mr.python.create_model(
    name=MODEL_NAME,
    #version=MODEL_VERSION,
    version = 3,
    description=model_description,
    metrics={
        "roc_auc": float(auc),
        "feature_view_version": int(FEATURE_VIEW_VERSION)
    }
)
registered_model.save(model_dir)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model/model.pkl: …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1279154/models/aurora_xgboost/3


Model(name: 'aurora_xgboost', version: 3)

In [49]:
# Save metrics from training


# 2. Add metrics
registered_model.add_metrics({
    "roc_auc": float(auc),
    "feature_view_version": FEATURE_VIEW_VERSION
})


AttributeError: 'Model' object has no attribute 'add_metrics'